In [1]:
# 自分の Google Drive をマウント
# https://qiita.com/uni-3/items/201aaa2708260cc790b8#drive内のディレクトリをマウントする220180920
from google.colab import drive
import os
drive.mount('/content/drive')

#@markdown ## 【データを保存するGoogleDriveのパス】
#@markdown ### GitHubリポジトリをcloneするパス
REPOSITORY_PEARENT_DIR = '/content/drive/My Drive/__datasets__/[CV][NLP]\u300C\u30BB\u30F3\u30BF\u30FC\u8A66\u9A13xml\u300D/annotate_img/datas/attribute/Experiment' #@param {type: "string"}
os.chdir(REPOSITORY_PEARENT_DIR)

Mounted at /content/drive


# RNN を使ったテキスト分類

In [2]:
# !pip install -q tf-nightly
# import tensorflow_datasets as tfds
# !pip3 list | grep tensorflow
# import tensorflow as tf

!pip3 install sentencepiece
import sentencepiece as spm

import json
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPUを使うために必要
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     |████████████████████████████████| 1.2MB 4.3MB/s 


In [3]:
# import matplotlib.pyplot as plt

# def plot_graphs(history, metric):
#   plt.plot(history.history[metric])
#   plt.plot(history.history['val_'+metric], '')
#   plt.xlabel("Epochs")
#   plt.ylabel(metric)
#   plt.legend([metric, 'val_'+metric])
#   plt.show()

## Data


### load

In [4]:
sbj_names = 'Eigo' #@param [['Eigo'], ['Suugaku'], ['Eigo', 'Suugaku']]
sbj_names = sbj_names.split(',')
print(sbj_names)
elmnt_name = 'question'
attr_name = 'knowledge_type'   #@param ['knowledge_type', 'answer_type']
is_remove_xml = True #@param {type:"boolean"}

df = None
for sbj in sbj_names:
    print(sbj)
    attr_csv_path = f'../{sbj}_{attr_name}_ds.tsv'
    df_tmp = pd.read_csv(attr_csv_path, delimiter='\t')
    df = pd.concat([df, df_tmp])

df = df.reset_index(drop=True)
df = df.dropna()  # nan を削除
df

['Eigo']
Eigo


,Unnamed: 0,knowledge_type,<instruction/>,contents
0,0,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問１</label> <ansColumn id=""A1"">1</ansC..."
1,1,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問２</label> <ansColumn id=""A2"">2</ansC..."
2,2,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問３</label> <ansColumn id=""A3"">3</ansC..."
3,3,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問４</label> <ansColumn id=""A4"">4</ansC..."
4,4,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問５</label> <ansColumn id=""A5"">5</ansC..."
...,...,...,...,...
2630,2630,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問２</label> <data id=""D44"" type=""text..."
2631,2631,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問３</label> <data id=""D45"" type=""text..."
2632,2632,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問４</label> <data id=""D46"" type=""text..."
2633,2633,Other,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問５</label> <data id=""D47"" type=""text..."


In [5]:
## XMLタグ除去
if is_remove_xml:
    import re
    def remove_xml(xml_child_str):
        xml_child_str = re.sub('<.*?>', '', xml_child_str)
        return re.sub('</.*?>', '', xml_child_str)

    # test_idx = 200
    # print(df['contents'][test_idx])
    # print(remove_xml(df['contents'][test_idx]))
    df['contents'] = df['contents'].map(remove_xml)
df

,Unnamed: 0,knowledge_type,<instruction/>,contents
0,0,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問１</label> <ansColumn id=""A1"">1</ansC..."
1,1,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問２</label> <ansColumn id=""A2"">2</ansC..."
2,2,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問３</label> <ansColumn id=""A3"">3</ansC..."
3,3,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問４</label> <ansColumn id=""A4"">4</ansC..."
4,4,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問５</label> <ansColumn id=""A5"">5</ansC..."
...,...,...,...,...
2630,2630,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問２</label> <data id=""D44"" type=""text..."
2631,2631,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問３</label> <data id=""D45"" type=""text..."
2632,2632,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問４</label> <data id=""D46"" type=""text..."
2633,2633,Other,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問５</label> <data id=""D47"" type=""text..."


### Tokenize
SentencePiece を使用。
- タグ あり／なし

In [6]:
m_dir = 'model/SentencePiece'
os.makedirs(m_dir, exist_ok=True)
df.to_csv(f'{m_dir}/tmp.txt', sep='\t')

# arg_str = '--input={m_dir}/tmp.txt --model_prefix={m_dir}/m_user ' + '--user_defined_symbols=<sep>,<cls>' + ',<ansColumn/>,<label>' + ' --vocab_size=2000'
# spm.SentencePieceTrainer.train(arg_str)

spm.SentencePieceTrainer.train(f'--input={m_dir}/tmp.txt --model_prefix={m_dir}/m  --user_defined_symbols=<sep>,<cls>,<pad>   --vocab_size=2000')
sp = spm.SentencePieceProcessor()  # model_file='SentencePiece/test_model.model'

sp.load(f'{m_dir}/m.model')

True

In [7]:
# encode: text => id
# tokenized_tokens =  sp.encode_as_pieces('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
# print(tokenized_tokens)

# tokenized_ids = sp.encode_as_ids('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
# print(tokenized_ids)

# decoded_text = sp.decode(tokenized_ids)
# print(decoded_text)

In [8]:
# example_content = df['contents'][20]
# print(example_content, '\n', sp.encode_as_pieces(example_content))

In [9]:
# for index in encoded_string:
#   print('{} ----> {}'.format(index, encoder.decode([index])))

## Train 用データの準備

In [10]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>":0})

for inst, cont in zip(df['<instruction/>'], df['contents']):
#     try:
    tokens = sp.encode_as_pieces(inst + cont)
    for word in tokens:
            if word in word2index: continue
            word2index[word] = len(word2index)
#     except TypeError:
#         print(f'[Error] <instruction/> が nan です。')
#         print(f'    inst : {inst}')
#         print(f'    cont : {cont}')

print("vocab size : ", len(word2index))


vocab size :  2045


In [11]:
## set_dict から自動抽出する！
# attr_name = 'knowledge_type'  # 'answer_type'

categories = set()
for sbj in sbj_names:
    with open(f'../class_set/{sbj}-{elmnt_name}-{attr_name}.json') as f:
        categories |= set(json.load(f))   # sbj_names = ['Eigo', ]

# print(categories)

# categories = [
#     'sentence', 
#     'term_person', 'term_location', 'term_time', 'term_other',
#     'referenceSymbol',
#     'image_graph', 'image_photo', 'image_map', 'image_table', 'image_other',
#     'formula', 
#     'orthography',
#     'other',
#     # 組み合わせ系（仮追加）
#     '(symbol-sentence)*2', '(symbol-sentence)*3', '(symbol-sentence)*4', '(symbol-term_location)*3', '(symbol-term_other)*3', '(symbol-term_other)*3',
#     '(symbol-symbol)*4',
#     '(term_location-term_location-term_location)', 'term_location-term_location-term_location-term_location',
#     '(term_location-term_location-term_location)', '(term_location-term_location-term_location)',
#     '(term_other-term_other-term_other)',
#     'term_other-term_other-term_other',
#     'sentence-sentence',
#     'sentence-sentence-sentence',
#     'symbol-symbol',
#     'symbol-symbol-symbol',
#     'symbol-symbol-symbol-symbol',
#     'o(symbol-symbol-symbol-symbol)',
#     'o(symbol-symbol-symbol)',
# ]

categories = list(categories)
categories.sort()    # 入れないと、クラス番号が変わってしまい、再現実験ができないので注意？
print(categories)
print(len(categories))

['DIC_O', 'DIC_O, DIS_S, R_ENT', 'DIC_O, DIS_W', 'DIC_O, DIS_W, GK', 'DIC_O, EG', 'DIC_O, EG, DIS_W', 'DIC_O, EG, GK', 'DIC_O, GK', 'DIC_O, Other', 'DIC_O, SEL', 'DIC_O,DIS_W', 'DIC_O,GK', 'DIS_C, GK', 'DIS_O', 'DIS_O, DIS_W', 'DIS_O, DIS_W, GK', 'DIS_O, GK', 'DIS_O, Other', 'DIS_S', 'DIS_S, IC_O', 'DIS_S, R_ENT', 'DIS_S, R_ENT, IC_G', 'DIS_S, R_ENT, IC_M', 'DIS_S, R_ENT, IC_O', 'DIS_S, R_ENT, IC_T', 'DIS_S, R_ENT, R_SUM', 'DIS_S,R_ENT', 'DIS_S,R_ENT,R_SUM', 'DIS_S,R_QA', 'DIS_W', 'DIS_W, GK', 'DIS_W, R_ENT', 'DIS_W, R_ENT, IC_G', 'DIS_W, R_ENT, IC_G, IC_P', 'DIS_W, R_ENT, IC_M', 'DIS_W, R_ENT, IC_O', 'DIS_W, R_ENT, IC_O, IC_P', 'DIS_W, R_ENT, IC_P', 'DIS_W, R_ENT, IC_P, IC_T', 'DIS_W, R_ENT, IC_T', 'DIS_W, R_ENT, R_SUM', 'DIS_W, R_ENT, R_SUM, IC_O', 'DIS_W, R_QA', 'DIS_W,DIC_O', 'DIS_W,EG', 'DIS_W,GK', 'DIS_W,IC_P', 'DIS_W,R_ENT', 'DIS_W,R_ENT,IC_P', 'DIS_W,R_QA', 'DIS_W,R_SUM', 'DIS_W,R_SUM,IC_T', 'EG', 'EG, ', 'EG, DIS_S', 'EG, DIS_W', 'EG, DIS_W, GK', 'EG, GK', 'EG,DIS_W', 'EG,SEL'

In [12]:
## 系列の長さを揃えてバッチでまとめる
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)

def sentence2index(sentence):
    tokens = sp.encode_as_pieces(sentence)
    # print(tokens)
    return [word2index[w] for w in tokens]

def category2index(cat):
    return [cat2index[cat]]

index_datasets_c_xml_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for inst, cont, category in zip(df['<instruction/>'], df['contents'], df[attr_name]):
    index_c_xml = sentence2index(inst + cont)
    index_category = category2index(category)
    index_datasets_c_xml_tmp.append(index_c_xml)
    index_datasets_category.append(index_category)
    if max_len < len(index_c_xml):
        max_len = len(index_c_xml)

# 系列の長さを揃えるために短い系列にパディングを追加
# 後ろパディングだと正しく学習できなかったので、前パディング
index_datasets_c_xml = []
for c_xml in index_datasets_c_xml_tmp:
    for i in range(max_len - len(c_xml)):
        c_xml.insert(0, 0) # 前パディング
#     c_xml.append(0)　# 後ろパディング
    index_datasets_c_xml.append(c_xml)

train_x, test_x, train_y, test_y = train_test_split(index_datasets_c_xml, index_datasets_category, train_size=0.7)

# データをバッチでまとめるための関数
def train2batch(c_xml, category, batch_size=100):
    c_xml_batch = []
    category_batch = []
    c_xml_shuffle, category_shuffle = shuffle(c_xml, category)
    for i in range(0, len(c_xml), batch_size):
        c_xml_batch.append(c_xml_shuffle[i:i+batch_size])
        category_batch.append(category_shuffle[i:i+batch_size])
    return c_xml_batch, category_batch

## Model

In [13]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores  

    def load_weights(self, load_m_path='_logs/test/LSTM_classifier.pth',):
        if load_m_path is not None:
            param = torch.load(load_m_path)
            self.load_state_dict(param)
            print(f'[info] {load_m_path} loaded !')

    def save(self, save_f_path='_logs/test/LSTM_classifier.pth',):
        torch.save(self.state_dict(), save_f_path)

    
# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)

model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)

loss_function = nn.NLLLoss()
# SGDからAdamに変更。特に意味はなし
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
## モデルの保存場所を準備する。
import datetime
dt_now = datetime.datetime.now()
save_m_dir = os.path.join('_logs', dt_now.strftime('%Y-%m-%d_%Hh%Mm%Ss') + '_colab')
print('[Info] save_m_dir', save_m_dir)
save_m_path = os.path.join(save_m_dir, 'LSTM_classifier.pth')
os.makedirs(save_m_dir, exist_ok=True)

[Info] save_m_dir _logs/2021-06-16_10h00m03s_colab


In [15]:
## 実験設定などをメモする。
import json

param_list = {
    "sbj_names": sbj_names,
    "elmnt_name": elmnt_name,
    "attr_name": attr_name,
    "is_remove_xml": is_remove_xml
}

with open(os.path.join(save_m_dir, 'params_log.json'), 'w') as f:
    json.dump(param_list, f, ensure_ascii=False)

## Experiment Train

In [16]:
min_loss = 1.0
losses = []
for epoch in range(1000):
    all_loss = 0
    title_batch, category_batch = train2batch(train_x, train_y)
    for i in range(len(title_batch)):
        batch_loss = 0

        model.zero_grad()

        # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
        title_tensor = torch.tensor(title_batch[i], device=device)
        # category_tensor.size() = (batch_size × 1)なので、squeeze()
        category_tensor = torch.tensor(category_batch[i], device=device).squeeze()

        out = model(title_tensor)

        batch_loss = loss_function(out, category_tensor)
        batch_loss.backward()
        optimizer.step()

        all_loss += batch_loss.item()
        if min_loss > all_loss:
            model.save(save_m_path)

    print("epoch", epoch, "\t" , "loss", all_loss)
    if all_loss < 0.001: break
    if 0.009 < all_loss < 0.01:
        model.save(save_m_path_loss(0.01))
    if 0.09 < all_loss < 0.1:
        model.save(save_m_path_loss(0.1))
print("done.")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 78.18533205986023
epoch 1 	 loss 67.92081379890442
epoch 2 	 loss 65.84880638122559
epoch 3 	 loss 61.90516114234924
epoch 4 	 loss 54.90120482444763
epoch 5 	 loss 48.459089279174805
epoch 6 	 loss 43.73667931556702
epoch 7 	 loss 40.03215157985687
epoch 8 	 loss 36.823230624198914
epoch 9 	 loss 34.312376618385315
epoch 10 	 loss 31.812422156333923
epoch 11 	 loss 29.761302709579468
epoch 12 	 loss 27.934564113616943
epoch 13 	 loss 25.93952488899231
epoch 14 	 loss 23.564752101898193
epoch 15 	 loss 21.631917417049408
epoch 16 	 loss 19.73838710784912
epoch 17 	 loss 18.470475912094116
epoch 18 	 loss 17.15600895881653
epoch 19 	 loss 15.14820384979248
epoch 20 	 loss 13.516847312450409
epoch 21 	 loss 12.13250869512558
epoch 22 	 loss 11.467586308717728
epoch 23 	 loss 10.58800095319748
epoch 24 	 loss 9.385150372982025
epoch 25 	 loss 8.355066329240799
epoch 26 	 loss 7.407173126935959
epoch 27 	 loss 6.725616663694382
epoch 28 	 loss 5.911166429519653
epoch 29 	 lo

___

## Experiment Eval

In [21]:
test_num = len(test_x)
a = 0
negative_sample = [("Pred", "Ans")]
with torch.no_grad():
    title_batch, category_batch = train2batch(test_x, test_y)

    for i in range(len(title_batch)):
        title_tensor = torch.tensor(title_batch[i], device=device)
        category_tensor = torch.tensor(category_batch[i], device=device)

        out = model(title_tensor)
        _, predicts = torch.max(out, 1)
        for j, ans in enumerate(category_tensor):
            if predicts[j].item() == ans.item():
                a += 1
            else:
#                 print(predicts[j].item(), ans.item())
                negative_sample.append((predicts[j].item(), ans.item()))
print("predict : ", a / test_num)
# predict :  0.6967916854948034

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


predict :  0.4222503160556258


In [22]:
## 実験設定などをメモする。
import json

param_list = {
    "sbj_names": sbj_names,
    "elmnt_name": elmnt_name,
    "attr_name": attr_name,
    "is_remove_xml": is_remove_xml,
    "predict": a / test_num,
    "間違えた例" : negative_sample
}

with open(os.path.join(save_m_dir, 'params_log.json'), 'w') as f:
    json.dump(param_list, f, ensure_ascii=False, indent=4)

___

## 2つ以上の LSTM レイヤーを重ねる

Keras のリカレントレイヤーには、コンストラクタの `return_sequences` 引数でコントロールされる2つのモードがあります。

* それぞれのタイムステップの連続した出力のシーケンス全体（shape が `(batch_size, timesteps, output_features)` の3階テンソル）を返す。
* それぞれの入力シーケンスの最後の出力だけ（shape が `(batch_size, output_features)` の2階テンソル）を返す。

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

NameError: ignored

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# パディングなしのサンプルテキストの推論

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

In [ ]:
# パディングありのサンプルテキストの推論

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

[GRU レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)など既存のほかのレイヤーを調べてみましょう。

カスタム RNN の構築に興味があるのであれば、[Keras RNN ガイド](../../guide/keras/rnn.ipynb) を参照してください。